# Spark Individual Assignment

In [17]:
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

from pyspark.sql.functions import sum

In [18]:
hotelsDF = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .csv("hotel_bookings.csv")

In [19]:
from IPython.display import display, Markdown

hotelsDF.printSchema()
display(Markdown("This DataFrame has **%d rows**." % hotelsDF.count()))

root
 |-- hotel: string (nullable = true)
 |-- is_canceled: integer (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- arrival_date_year: integer (nullable = true)
 |-- arrival_date_month: string (nullable = true)
 |-- arrival_date_week_number: integer (nullable = true)
 |-- arrival_date_day_of_month: integer (nullable = true)
 |-- stays_in_weekend_nights: integer (nullable = true)
 |-- stays_in_week_nights: integer (nullable = true)
 |-- adults: integer (nullable = true)
 |-- children: string (nullable = true)
 |-- babies: integer (nullable = true)
 |-- meal: string (nullable = true)
 |-- country: string (nullable = true)
 |-- market_segment: string (nullable = true)
 |-- distribution_channel: string (nullable = true)
 |-- is_repeated_guest: integer (nullable = true)
 |-- previous_cancellations: integer (nullable = true)
 |-- previous_bookings_not_canceled: integer (nullable = true)
 |-- reserved_room_type: string (nullable = true)
 |-- assigned_room_type: string (nullab

This DataFrame has **119390 rows**.

In [20]:
hotelsDF.cache() # optimization to make the processing faster
hotelsDF.sample(False, 0.1).take(2)

[Row(hotel='Resort Hotel', is_canceled=0, lead_time=37, arrival_date_year=2015, arrival_date_month='July', arrival_date_week_number=27, arrival_date_day_of_month=1, stays_in_weekend_nights=0, stays_in_week_nights=4, adults=2, children='0', babies=0, meal='BB', country='PRT', market_segment='Offline TA/TO', distribution_channel='TA/TO', is_repeated_guest=0, previous_cancellations=0, previous_bookings_not_canceled=0, reserved_room_type='E', assigned_room_type='E', booking_changes=0, deposit_type='No Deposit', agent='8', company='NULL', days_in_waiting_list=0, customer_type='Contract', adr=97.5, required_car_parking_spaces=0, total_of_special_requests=0, reservation_status='Check-Out', reservation_status_date='2015-07-05'),
 Row(hotel='Resort Hotel', is_canceled=0, lead_time=72, arrival_date_year=2015, arrival_date_month='July', arrival_date_week_number=27, arrival_date_day_of_month=1, stays_in_weekend_nights=2, stays_in_week_nights=4, adults=2, children='0', babies=0, meal='BB', country=

### Data entities, metrics and dimensions

I've identified the following elements:

* **Entities:** Resort Hotel and City Hotel
* **Metrics:** Total number of bookings, Cancellation Rate
* **Dimensions:** New vs. Old Customers, Children vs. No children...

### Column categorization

The following could be a potential column categorization:

* **Timing related columns:** *arrival_date_year*, *arrival_date_month*, *arrival_date_week_number*, *arrival_date_day_of_month*, *lead_time*
* **Guest related columns:** *is_repeated_guest*, *company*, *babies*, *children*, *deposit_type*, *customer_type*, *booking_changes*
* **Issue related columns:** *is_cancelled*

## Columns groups basic profiling to better understand our data set
### A. Timing related columns basic profiling

In [21]:
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, lit


print ("Summary of columns arrival_date_year, arrival_date_month, arrival_date_day_of_month, arrival_date_week_number, lead_time:")
hotelsDF.select("arrival_date_year","arrival_date_month","arrival_date_day_of_month","arrival_date_week_number", "lead_time").summary().show()

print("Checking for nulls on columns arrival_date_year, arrival_date_month, arrival_date_day_of_month, arrival_date_week_number, lead_time:")
hotelsDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["arrival_date_year","arrival_date_month","arrival_date_day_of_month","arrival_date_week_number", "lead_time"]]).show()

print("Checking amount of distinct values in columns arrival_date_year, arrival_date_month, arrival_date_day_of_month, arrival_date_week_number, lead_time:")
hotelsDF.select([countDistinct(c).alias(c) for c in ["arrival_date_year","arrival_date_month","arrival_date_day_of_month","arrival_date_week_number", "lead_time"]]).show()

print ("Most and least frequent occurrences for arrival_date_day_of_month and arrival_date_week_number columns:")
dayofMonthOccurrencesDF = hotelsDF.groupBy("arrival_date_day_of_month").agg(count(lit(1)).alias("Total"))
weekNumberDF = hotelsDF.groupBy("arrival_date_week_number").agg(count(lit(1)).alias("Total"))

leastFreqDayOfMonth    = dayofMonthOccurrencesDF.orderBy(col("Total").asc()).first()
mostFreqDayOfMonth     = dayofMonthOccurrencesDF.orderBy(col("Total").desc()).first()
leastFreqWeek     = weekNumberDF.orderBy(col("Total").asc()).first()
mostFreqWeek      = weekNumberDF.orderBy(col("Total").desc()).first()

display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("leastFreqDayOfMonth", "mostFreqDayOfMonth", "leastFreqWeekNumber", "mostFreqWeekNumber", \
       "%d (%d occurrences)" % (leastFreqDayOfMonth["arrival_date_day_of_month"], leastFreqDayOfMonth["Total"]), \
       "%d (%d occurrences)" % (mostFreqDayOfMonth["arrival_date_day_of_month"], mostFreqDayOfMonth["Total"]), \
       "%d (%d occurrences)" % (leastFreqWeek["arrival_date_week_number"], leastFreqWeek["Total"]), \
       "%d (%d occurrences)" % (mostFreqWeek["arrival_date_week_number"], mostFreqWeek["Total"]))))

Summary of columns arrival_date_year, arrival_date_month, arrival_date_day_of_month, arrival_date_week_number, lead_time:
+-------+------------------+------------------+-------------------------+------------------------+------------------+
|summary| arrival_date_year|arrival_date_month|arrival_date_day_of_month|arrival_date_week_number|         lead_time|
+-------+------------------+------------------+-------------------------+------------------------+------------------+
|  count|            119390|            119390|                   119390|                  119390|            119390|
|   mean| 2016.156554150264|              null|       15.798241058715135|       27.16517296255968|104.01141636652986|
| stddev|0.7074759445220408|              null|        8.780829470578343|      13.605138355497665| 106.8630970479881|
|    min|              2015|             April|                        1|                       1|                 0|
|    25%|              2016|              null|     


| leastFreqDayOfMonth | mostFreqDayOfMonth | leastFreqWeekNumber | mostFreqWeekNumber |
|----|----|----|----|
| 31 (2208 occurrences) | 17 (4406 occurrences) | 51 (933 occurrences) | 33 (3580 occurrences) |


### B. Guest related columns basic profiling

In [22]:
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first

print ("Summary of columns is_repeated_guest, company, babies, children, deposit_type, country, is_canceled:")
hotelsDF.select("is_repeated_guest", "company", "babies", "children", "deposit_type", "country", "is_canceled").summary().show()

print("Checking for nulls on columns is_repeated_guest, company, babies, children, deposit_type, country, is_canceled:")
hotelsDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["is_repeated_guest", "company", "babies", "children", "deposit_type", "country", "is_canceled"]]).show()

print("Checking amount of distinct values in columns is_repeated_guest, company, babies, children, deposit_type, country, is_canceled:")
hotelsDF.select([countDistinct(c).alias(c) for c in ["is_repeated_guest", "company", "babies", "children", "deposit_type", "country", "is_canceled"]]).show()

Summary of columns is_repeated_guest, company, babies, children, deposit_type, country, is_canceled:
+-------+-------------------+------------------+--------------------+-------------------+------------+-------+-------------------+
|summary|  is_repeated_guest|           company|              babies|           children|deposit_type|country|        is_canceled|
+-------+-------------------+------------------+--------------------+-------------------+------------+-------+-------------------+
|  count|             119390|            119390|              119390|             119390|      119390| 119390|             119390|
|   mean|0.03191222045397437|189.26673532440782|0.007948739425412514|0.10388990333874994|        null|   null|0.37041628277075134|
| stddev|0.17576714541065672| 131.6550146385122|  0.0974361913012642| 0.3985614447864427|        null|   null|0.48291822659259803|
|    min|                  0|                10|                   0|                  0|  No Deposit|    ABW|   

## Overall Analysis

In [23]:
from pyspark.sql.functions import count, round

totalbookings = hotelsDF.count()

BookingbyYear = hotelsDF.select("arrival_date_year")\
.groupBy("arrival_date_year")\
.agg(count(lit(1)).alias("Bookings"), \
     (count(lit(1))/totalbookings*100).alias("Ratio"))\
.orderBy(col("arrival_date_year"))\
     .select("arrival_date_year","Bookings",round("Ratio",2).alias("RoundedRatio"))

CancellationbyYear = hotelsDF.where(col("is_canceled")==1)\
.select("arrival_date_year")\
.groupBy("arrival_date_year")\
.agg(count(lit(1)).alias("Cancellations"), \
     (count(lit(1))/totalbookings*100).alias("Ratio"))\
.orderBy(col("arrival_date_year"))\
     .select("arrival_date_year","Cancellations",round("Ratio",2).alias("RoundedRatio"))

BookingbyYear\
.join(CancellationbyYear, "arrival_date_year")\
.withColumn("Cancellation Ratio", round(col("Cancellations")/col("Bookings")*100,2))\
.orderBy(col("arrival_date_year")).show()

# Improved code without needing to join
hotelsDF.select("arrival_date_year","is_canceled")\
.groupBy("arrival_date_year")\
.agg(count(lit(1)).alias("Bookings"), \
     sum("is_canceled").alias("Cancellations"))\
.withColumn("Cancellation Ratio", round(col("Cancellations")/col("Bookings")*100,2))\
.orderBy(col("arrival_date_year"))\
     .select("arrival_date_year","Bookings","Cancellations", "Cancellation Ratio").show()

+-----------------+--------+------------+-------------+------------+------------------+
|arrival_date_year|Bookings|RoundedRatio|Cancellations|RoundedRatio|Cancellation Ratio|
+-----------------+--------+------------+-------------+------------+------------------+
|             2015|   21996|       18.42|         8142|        6.82|             37.02|
|             2016|   56707|        47.5|        20337|       17.03|             35.86|
|             2017|   40687|       34.08|        15745|       13.19|              38.7|
+-----------------+--------+------------+-------------+------------+------------------+

+-----------------+--------+-------------+------------------+
|arrival_date_year|Bookings|Cancellations|Cancellation Ratio|
+-----------------+--------+-------------+------------------+
|             2015|   21996|         8142|             37.02|
|             2016|   56707|        20337|             35.86|
|             2017|   40687|        15745|              38.7|
+----------

## Overall Cancellation Rate

In [24]:
from pyspark.sql.functions import count, round

cancelledDF = hotelsDF\
   .withColumn("Status", when(col("is_canceled")==1,"Cancelled")\
                               .otherwise("Not Cancelled"))

hotelsDF.select("reservation_status")\
.groupBy("reservation_status")\
.agg(count("reservation_status").alias("Count"), \
     (count("reservation_status")/totalbookings*100).alias("Ratio"))\
.orderBy("reservation_status")\
     .select("reservation_status","Count",round("Ratio",2).alias("RoundedRatio")).show()

+------------------+-----+------------+
|reservation_status|Count|RoundedRatio|
+------------------+-----+------------+
|          Canceled|43017|       36.03|
|         Check-Out|75166|       62.96|
|           No-Show| 1207|        1.01|
+------------------+-----+------------+



## Ratio of reservation cancellation by resort vs. city

In [25]:
totalcancelled = hotelsDF.where(col('is_canceled') == 1).count()

display(Markdown("**'Number and Ratio of cancellations, City vs. Resort hotel:"))

HotelTypeCancellation = hotelsDF.where(col("is_canceled")==1)\
.select("hotel")\
.groupBy("hotel")\
.agg(count("hotel").alias("Cancellations"), \
     (count("hotel")/totalcancelled*100).alias("Ratio"))\
     .select("hotel","Cancellations",round("Ratio",2).alias("RoundedRatio"))

HotelTypeBooking = hotelsDF\
.select("hotel")\
.groupBy("hotel")\
.agg(count("hotel").alias("Bookings"), \
     (count("hotel")/totalbookings*100).alias("Ratio"))\
     .select("hotel","Bookings",round("Ratio",2).alias("RoundedRatio"))

HotelTypeBooking\
.join(HotelTypeCancellation, "hotel")\
.withColumn("Cancellation Ratio", round(col("Cancellations")/col("Bookings")*100,2))\
.orderBy(col("hotel")).show()

#Improved code without needing to join
hotelsDF.select("hotel", "is_canceled")\
.groupby("hotel")\
.agg(count(lit(1)).alias("Bookings"),\
    sum("is_canceled").alias("Cancellations"))\
.withColumn("Cancellation Ratio", round(col("Cancellations")/col("Bookings")*100,2))\
.orderBy(col("Cancellation Ratio").desc())\
     .select("hotel","Bookings","Cancellations", "Cancellation Ratio").show()

**'Number and Ratio of cancellations, City vs. Resort hotel:

+------------+--------+------------+-------------+------------+------------------+
|       hotel|Bookings|RoundedRatio|Cancellations|RoundedRatio|Cancellation Ratio|
+------------+--------+------------+-------------+------------+------------------+
|  City Hotel|   79330|       66.45|        33102|       74.85|             41.73|
|Resort Hotel|   40060|       33.55|        11122|       25.15|             27.76|
+------------+--------+------------+-------------+------------+------------------+

+------------+--------+-------------+------------------+
|       hotel|Bookings|Cancellations|Cancellation Ratio|
+------------+--------+-------------+------------------+
|  City Hotel|   79330|        33102|             41.73|
|Resort Hotel|   40060|        11122|             27.76|
+------------+--------+-------------+------------------+



## Cancellation Analysis - City Hotel

In [26]:
hotelsDF.where(col("is_canceled")==1)\
.select("country")\
.groupBy("country")\
.agg(count("country").alias("Count"), \
     (count("country")/totalcancelled*100).alias("Ratio"))\
     .select("country","Count",round("Ratio",2).alias("RoundedRatio")).show()

+-------+-----+------------+
|country|Count|RoundedRatio|
+-------+-----+------------+
|    POL|  215|        0.49|
|    LVA|    9|        0.02|
|    ZMB|    1|         0.0|
|    BRA|  830|        1.88|
|    ARM|    2|         0.0|
|    MOZ|   19|        0.04|
|    JOR|    3|        0.01|
|    FRA| 1934|        4.37|
|    URY|    9|        0.02|
|    GIB|   11|        0.02|
|    ETH|    1|         0.0|
|     CN|  254|        0.57|
|    ITA| 1333|        3.01|
|    UKR|   20|        0.05|
|    GHA|    2|         0.0|
|    SEN|    8|        0.02|
|    HRV|   25|        0.06|
|    QAT|   11|        0.02|
|    GBR| 2453|        5.55|
|    ARE|   43|         0.1|
+-------+-----+------------+
only showing top 20 rows



In [27]:
countriesDF = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .csv("CountryMapping.csv")

countriesDF.show()

+-------------------+-------+-------+------------+-------------+--------+--------------------+-------------------+-----------+---------------+------------------------+
|               name|alpha-2|alpha-3|country-code|   iso_3166-2|  region|          sub-region|intermediate-region|region-code|sub-region-code|intermediate-region-code|
+-------------------+-------+-------+------------+-------------+--------+--------------------+-------------------+-----------+---------------+------------------------+
|        Afghanistan|     AF|    AFG|           4|ISO 3166-2:AF|    Asia|       Southern Asia|               null|        142|             34|                    null|
|      Åland Islands|     AX|    ALA|         248|ISO 3166-2:AX|  Europe|     Northern Europe|               null|        150|            154|                    null|
|            Albania|     AL|    ALB|           8|ISO 3166-2:AL|  Europe|     Southern Europe|               null|        150|             39|                  

In [28]:
countriesDF = countriesDF.withColumnRenamed("alpha-3", "alpha3")

combinedDF = \
  hotelsDF\
     .join(countriesDF, hotelsDF.country == countriesDF.alpha3)\

combinedDF.cache()

BookingbyRegion = combinedDF.where(col("hotel") == "City Hotel")\
.select("region")\
.groupBy("region")\
.agg(count("region").alias("Booking Count")) \
.orderBy(col("Booking Count").desc())\
     .select("region","Booking Count")

CancellationbyRegion = combinedDF.where((col("is_canceled")==1) & (col("hotel") == "City Hotel"))\
.select("region")\
.groupBy("region")\
.agg(count("region").alias("Cancellation Count")) \
.orderBy(col("Cancellation Count").desc())\
     .select("region","Cancellation Count")

display(Markdown("Number of cancellations ratio per region:"))
BookingbyRegion\
.join(CancellationbyRegion, "region")\
.withColumn("Cancellation Ratio", round(col("Cancellation Count")/col("Booking Count")*100,2))\
.orderBy(col("Cancellation Ratio").desc()).show()

# Improved code without needing to join
combinedDF.where((col("hotel") == "City Hotel") & col("region").isNotNull())\
.select("region", "is_canceled")\
.groupby("region")\
.agg(count(lit(1)).alias("Bookings"),\
    sum("is_canceled").alias("Cancellations"))\
.withColumn("Cancellation Ratio", round(col("Cancellations")/col("Bookings")*100,2))\
.orderBy(col("Bookings").desc())\
     .select("region","Bookings","Cancellations", "Cancellation Ratio").show()

Number of cancellations ratio per region:

+--------+-------------+------------------+------------------+
|  region|Booking Count|Cancellation Count|Cancellation Ratio|
+--------+-------------+------------------+------------------+
|  Africa|          940|               418|             44.47|
|  Europe|        70649|             29980|             42.44|
|    Asia|         2805|              1132|             40.36|
|Americas|         3934|              1292|             32.84|
| Oceania|          404|                99|              24.5|
+--------+-------------+------------------+------------------+

+--------+--------+-------------+------------------+
|  region|Bookings|Cancellations|Cancellation Ratio|
+--------+--------+-------------+------------------+
|  Europe|   70649|        29980|             42.44|
|Americas|    3934|         1292|             32.84|
|    Asia|    2805|         1132|             40.36|
|  Africa|     940|          418|             44.47|
| Oceania|     404|           99|              24.5|
+-------

In [29]:
totalcancelledCity = hotelsDF.where((col("is_canceled")==1) & (col("hotel") == "City Hotel")).count()

# lead time <= 7 days, 1 week
# lead time between 7 and 90 days, within 3 months
# lead time between 90 and 180 days, within 6 months
# lead time between 180 and 365 days, within a year
# lead time of more than 365 days, more than 1 year
display(Markdown("Number of cancellations based on how much in advance the booking was made:"))
leadTimeDF = hotelsDF\
   .withColumn("booking_time", when(col("lead_time")<=7,"Within 1 Week")\
                               .when((col("lead_time")>7) & (col("lead_time")<=90),"Within 3 Months")\
                               .when((col("lead_time")>90) & (col("lead_time")<=180),"Within 6 Months")\
                               .when((col("lead_time")>180) & (col("lead_time")<=365),"Within 1 Year")\
                               .otherwise("More than 1 Year"))

leadTimeDF.cache()

leadTimeDF.where((col("is_canceled")==1) & (col("hotel") == "City Hotel"))\
.select("booking_time")\
.groupBy("booking_time")\
.agg(count("booking_time").alias("Cancellations"), \
     (count("booking_time")/totalcancelledCity*100).alias("Ratio"))\
.orderBy(col("Ratio").desc())\
     .select("booking_time","Cancellations",round("Ratio",2).alias("Ratio")).show()

# Improved code
leadTimeDF.where(col("hotel") == "City Hotel")\
.select("booking_time", "is_canceled")\
.groupby("booking_time")\
.agg(count(lit(1)).alias("Bookings"),\
    sum("is_canceled").alias("Cancellations"))\
.withColumn("Cancellation Ratio", round(col("Cancellations")/col("Bookings")*100,2))\
.orderBy(col("Cancellation Ratio").desc())\
     .select("booking_time","Bookings","Cancellations", "Cancellation Ratio").show()

display(Markdown("Number of cancellations based on deposit type:"))
hotelsDF.where((col("is_canceled")==1) & (col("hotel") == "City Hotel"))\
.select("deposit_type")\
.groupBy("deposit_type")\
.agg(count("deposit_type").alias("Cancellations"), \
     (count("deposit_type")/totalcancelledCity*100).alias("Ratio"))\
.orderBy(col("Ratio").desc())\
     .select("deposit_type","Cancellations",round("Ratio",2).alias("Ratio")).show()

# Improved code
hotelsDF.where(col("hotel") == "City Hotel")\
.select("deposit_type", "is_canceled")\
.groupby("deposit_type")\
.agg(count(lit(1)).alias("Bookings"),\
    sum("is_canceled").alias("Cancellations"))\
.withColumn("Cancellation Ratio", round(col("Cancellations")/col("Bookings")*100,2))\
.orderBy(col("Cancellation Ratio").desc())\
     .select("deposit_type","Bookings","Cancellations", "Cancellation Ratio").show()

display(Markdown("Number of cancellations, new vs. returning customer:"))
guestDF = hotelsDF\
   .withColumn("Customer Type", when(col("is_repeated_guest")==1,"Returning Customer")\
                               .otherwise("New Customer"))

guestDF.where((col("is_canceled")==1) & (col("hotel") == "City Hotel"))\
.select("Customer Type")\
.groupBy("Customer Type")\
.agg(count("Customer Type").alias("Count"), \
     (count("Customer Type")/totalcancelledCity*100).alias("Ratio"))\
.orderBy(col("Ratio").desc())\
     .select("Customer Type","Count",round("Ratio",2).alias("Ratio")).show()

# Improved code
guestDF.where(col("hotel") == "City Hotel")\
.select("Customer Type", "is_canceled")\
.groupby("Customer Type")\
.agg(count(lit(1)).alias("Bookings"),\
    sum("is_canceled").alias("Cancellations"))\
.withColumn("Cancellation Ratio", round(col("Cancellations")/col("Bookings")*100,2))\
.orderBy(col("Cancellation Ratio").desc())\
     .select("Customer Type","Bookings","Cancellations", "Cancellation Ratio").show()

# if "company" column does not have a company code and is therefore empty, consider as personal travel
display(Markdown("Number of cancellations, traveling for leisure vs. work:"))
companyDF = hotelsDF\
   .withColumn("Booking Reason", when(col("company")=="NULL","Personal")\
                               .otherwise("Work"))

companyDF.where((col("is_canceled")==1) & (col("hotel") == "City Hotel"))\
.select("Booking Reason")\
.groupBy("Booking Reason")\
.agg(count("Booking Reason").alias("Count"), \
     (count("Booking Reason")/totalcancelledCity*100).alias("Ratio"))\
.orderBy(col("Ratio").desc())\
     .select("Booking Reason","Count",round("Ratio",2).alias("Ratio")).show()

# Improved code
companyDF.where(col("hotel") == "City Hotel")\
.select("Booking Reason", "is_canceled")\
.groupby("Booking Reason")\
.agg(count(lit(1)).alias("Bookings"),\
    sum("is_canceled").alias("Cancellations"))\
.withColumn("Cancellation Ratio", round(col("Cancellations")/col("Bookings")*100,2))\
.orderBy(col("Cancellation Ratio").desc())\
     .select("Booking Reason","Bookings","Cancellations", "Cancellation Ratio").show()

# Columns "children" and "babies" are 0, consider no kids, otherwise with kids
# There are 3 rows wtih NAs for "children" column, have chosen to not include those.
display(Markdown("Number of cancellations, traveling with children vs. not:"))
childrenDF = hotelsDF\
   .withColumn("Children", when((col("children")==0) & (col("babies")==0),"No Kids")\
                               .otherwise("With Kids"))

childrenDF.where((col("is_canceled")==1) & (col("hotel") == "City Hotel"))\
.select("Children")\
.groupBy("Children")\
.agg(count("Children").alias("Count"), \
     (count("Children")/totalcancelledCity*100).alias("Ratio"))\
.orderBy(col("Ratio").desc())\
     .select("Children","Count",round("Ratio",2).alias("Ratio")).show()

# Improved code
childrenDF.where(col("hotel") == "City Hotel")\
.select("Children", "is_canceled")\
.groupby("Children")\
.agg(count(lit(1)).alias("Bookings"),\
    sum("is_canceled").alias("Cancellations"))\
.withColumn("Cancellation Ratio", round(col("Cancellations")/col("Bookings")*100,2))\
.orderBy(col("Cancellation Ratio").desc())\
     .select("Children","Bookings","Cancellations", "Cancellation Ratio").show()

Number of cancellations based on how much in advance the booking was made:

+----------------+-------------+-----+
|    booking_time|Cancellations|Ratio|
+----------------+-------------+-----+
| Within 3 Months|        12180| 36.8|
|   Within 1 Year|         8934|26.99|
| Within 6 Months|         8746|26.42|
|More than 1 Year|         1923| 5.81|
|   Within 1 Week|         1319| 3.98|
+----------------+-------------+-----+

+----------------+--------+-------------+------------------+
|    booking_time|Bookings|Cancellations|Cancellation Ratio|
+----------------+--------+-------------+------------------+
|More than 1 Year|    2704|         1923|             71.12|
|   Within 1 Year|   14244|         8934|             62.72|
| Within 6 Months|   18223|         8746|             47.99|
| Within 3 Months|   33351|        12180|             36.52|
|   Within 1 Week|   10808|         1319|              12.2|
+----------------+--------+-------------+------------------+



Number of cancellations based on deposit type:

+------------+-------------+-----+
|deposit_type|Cancellations|Ratio|
+------------+-------------+-----+
|  No Deposit|        20244|61.16|
|  Non Refund|        12844| 38.8|
|  Refundable|           14| 0.04|
+------------+-------------+-----+

+------------+--------+-------------+------------------+
|deposit_type|Bookings|Cancellations|Cancellation Ratio|
+------------+--------+-------------+------------------+
|  Non Refund|   12868|        12844|             99.81|
|  Refundable|      20|           14|              70.0|
|  No Deposit|   66442|        20244|             30.47|
+------------+--------+-------------+------------------+



Number of cancellations, new vs. returning customer:

+------------------+-----+-----+
|     Customer Type|Count|Ratio|
+------------------+-----+-----+
|      New Customer|32661|98.67|
|Returning Customer|  441| 1.33|
+------------------+-----+-----+

+------------------+--------+-------------+------------------+
|     Customer Type|Bookings|Cancellations|Cancellation Ratio|
+------------------+--------+-------------+------------------+
|      New Customer|   77298|        32661|             42.25|
|Returning Customer|    2032|          441|              21.7|
+------------------+--------+-------------+------------------+



Number of cancellations, traveling for leisure vs. work:

+--------------+-----+-----+
|Booking Reason|Count|Ratio|
+--------------+-----+-----+
|      Personal|32318|97.63|
|          Work|  784| 2.37|
+--------------+-----+-----+

+--------------+--------+-------------+------------------+
|Booking Reason|Bookings|Cancellations|Cancellation Ratio|
+--------------+--------+-------------+------------------+
|      Personal|   75641|        32318|             42.73|
|          Work|    3689|          784|             21.25|
+--------------+--------+-------------+------------------+



Number of cancellations, traveling with children vs. not:

+---------+-----+-----+
| Children|Count|Ratio|
+---------+-----+-----+
|  No Kids|31236|94.36|
|With Kids| 1866| 5.64|
+---------+-----+-----+

+---------+--------+-------------+------------------+
| Children|Bookings|Cancellations|Cancellation Ratio|
+---------+--------+-------------+------------------+
|  No Kids|   73923|        31236|             42.25|
|With Kids|    5407|         1866|             34.51|
+---------+--------+-------------+------------------+



## Cancellation Analysis - Resort Hotel

In [30]:
BookingbyRegion = combinedDF.where(col("hotel") == "Resort Hotel")\
.select("region")\
.groupBy("region")\
.agg(count("region").alias("Booking Count")) \
.orderBy(col("Booking Count").desc())\
     .select("region","Booking Count")

CancellationbyRegion = combinedDF.where((col("is_canceled")==1) & (col("hotel") == "Resort Hotel"))\
.select("region")\
.groupBy("region")\
.agg(count("region").alias("Cancellation Count")) \
.orderBy(col("Cancellation Count").desc())\
     .select("region","Cancellation Count")

display(Markdown("Number of cancellations ratio per region:"))
BookingbyRegion\
.join(CancellationbyRegion, "region")\
.withColumn("Cancellation Ratio", round(col("Cancellation Count")/col("Booking Count")*100,2))\
.orderBy(col("Cancellation Ratio").desc()).show()

# Improved code
combinedDF.where(col("hotel") == "Resort Hotel")\
.select("region", "is_canceled")\
.groupby("region")\
.agg(count(lit(1)).alias("Bookings"),\
    sum("is_canceled").alias("Cancellations"))\
.withColumn("Cancellation Ratio", round(col("Cancellations")/col("Bookings")*100,2))\
.orderBy(col("Bookings").desc())\
     .select("region","Bookings","Cancellations", "Cancellation Ratio").show()

Number of cancellations ratio per region:

+--------+-------------+------------------+------------------+
|  region|Booking Count|Cancellation Count|Cancellation Ratio|
+--------+-------------+------------------+------------------+
|  Africa|          174|                61|             35.06|
|  Europe|        37177|             10632|              28.6|
|    Asia|          384|                79|             20.57|
|Americas|         1048|               193|             18.42|
| Oceania|          103|                16|             15.53|
+--------+-------------+------------------+------------------+

+--------+--------+-------------+------------------+
|  region|Bookings|Cancellations|Cancellation Ratio|
+--------+--------+-------------+------------------+
|  Europe|   37177|        10632|              28.6|
|Americas|    1048|          193|             18.42|
|    Asia|     384|           79|             20.57|
|  Africa|     174|           61|             35.06|
| Oceania|     103|           16|             15.53|
+-------

In [31]:
totalcancelledResort = hotelsDF.where((col("is_canceled")==1) & (col("hotel") == "Resort Hotel")).count()

# lead time <= 7 days, 1 week
# lead time between 7 and 90 days, within 3 months
# lead time between 90 and 180 days, within 6 months
# lead time between 180 and 365 days, within a year
# lead time of more than 365 days, more than 1 year
display(Markdown("Number of cancellations based on how much in advance the booking was made:"))
leadTimeDF = hotelsDF\
   .withColumn("booking_time", when(col("lead_time")<=7,"Within 1 Week")\
                               .when((col("lead_time")>7) & (col("lead_time")<=90),"Within 3 Months")\
                               .when((col("lead_time")>90) & (col("lead_time")<=180),"Within 6 Months")\
                               .when((col("lead_time")>180) & (col("lead_time")<=365),"Within 1 Year")\
                               .otherwise("More than 1 Year"))

leadTimeDF.cache()

leadTimeDF.where((col("is_canceled")==1) & (col("hotel") == "Resort Hotel"))\
.select("booking_time")\
.groupBy("booking_time")\
.agg(count("booking_time").alias("Cancellations"), \
     (count("booking_time")/totalcancelledResort*100).alias("Ratio"))\
.orderBy(col("Ratio").desc())\
     .select("booking_time","Cancellations",round("Ratio",2).alias("Ratio")).show()

# Improved code
leadTimeDF.where(col("hotel") == "Resort Hotel")\
.select("booking_time", "is_canceled")\
.groupby("booking_time")\
.agg(count(lit(1)).alias("Bookings"),\
    sum("is_canceled").alias("Cancellations"))\
.withColumn("Cancellation Ratio", round(col("Cancellations")/col("Bookings")*100,2))\
.orderBy(col("Cancellation Ratio").desc())\
     .select("booking_time","Bookings","Cancellations", "Cancellation Ratio").show()

display(Markdown("Number of cancellations based on deposit type:"))
hotelsDF.where((col("is_canceled")==1) & (col("hotel") == "Resort Hotel"))\
.select("deposit_type")\
.groupBy("deposit_type")\
.agg(count("deposit_type").alias("Cancellations"), \
     (count("deposit_type")/totalcancelledResort*100).alias("Ratio"))\
.orderBy(col("Ratio").desc())\
     .select("deposit_type","Cancellations",round("Ratio",2).alias("Ratio")).show()

# Improved code
hotelsDF.where(col("hotel") == "Resort Hotel")\
.select("deposit_type", "is_canceled")\
.groupby("deposit_type")\
.agg(count(lit(1)).alias("Bookings"),\
    sum("is_canceled").alias("Cancellations"))\
.withColumn("Cancellation Ratio", round(col("Cancellations")/col("Bookings")*100,2))\
.orderBy(col("Cancellation Ratio").desc())\
     .select("deposit_type","Bookings","Cancellations", "Cancellation Ratio").show()

display(Markdown("Number of cancellations, new vs. returning customer:"))
guestDF = hotelsDF\
   .withColumn("Customer Type", when(col("is_repeated_guest")==1,"Returning Customer")\
                               .otherwise("New Customer"))

guestDF.where((col("is_canceled")==1) & (col("hotel") == "Resort Hotel"))\
.select("Customer Type")\
.groupBy("Customer Type")\
.agg(count("Customer Type").alias("Count"), \
     (count("Customer Type")/totalcancelledResort*100).alias("Ratio"))\
.orderBy(col("Ratio").desc())\
     .select("Customer Type","Count",round("Ratio",2).alias("Ratio")).show()

# Improved code
guestDF.where(col("hotel") == "Resort Hotel")\
.select("Customer Type", "is_canceled")\
.groupby("Customer Type")\
.agg(count(lit(1)).alias("Bookings"),\
    sum("is_canceled").alias("Cancellations"))\
.withColumn("Cancellation Ratio", round(col("Cancellations")/col("Bookings")*100,2))\
.orderBy(col("Cancellation Ratio").desc())\
     .select("Customer Type","Bookings","Cancellations", "Cancellation Ratio").show()

# if "company" column does not have a company code and is therefore empty, consider as personal travel
display(Markdown("Number of cancellations, traveling for leisure vs. work:"))
companyDF = hotelsDF\
   .withColumn("Booking Reason", when(col("company")=="NULL","Personal")\
                               .otherwise("Work"))

companyDF.where((col("is_canceled")==1) & (col("hotel") == "Resort Hotel"))\
.select("Booking Reason")\
.groupBy("Booking Reason")\
.agg(count("Booking Reason").alias("Count"), \
     (count("Booking Reason")/totalcancelledResort*100).alias("Ratio"))\
.orderBy(col("Ratio").desc())\
     .select("Booking Reason","Count",round("Ratio",2).alias("Ratio")).show()

# Improved code
companyDF.where(col("hotel") == "Resort Hotel")\
.select("Booking Reason", "is_canceled")\
.groupby("Booking Reason")\
.agg(count(lit(1)).alias("Bookings"),\
    sum("is_canceled").alias("Cancellations"))\
.withColumn("Cancellation Ratio", round(col("Cancellations")/col("Bookings")*100,2))\
.orderBy(col("Cancellation Ratio").desc())\
     .select("Booking Reason","Bookings","Cancellations", "Cancellation Ratio").show()

# Columns "children" and "babies" are 0, consider no kids, otherwise with kids
# There are 3 rows wtih NAs for "children" column, have chosen to not include those.
display(Markdown("Number of cancellations, traveling with children vs. not:"))
childrenDF = hotelsDF\
   .withColumn("Children", when((col("children")==0) & (col("babies")==0),"No Kids")\
                               .otherwise("With Kids"))

childrenDF.where((col("is_canceled")==1) & (col("hotel") == "Resort Hotel"))\
.select("Children")\
.groupBy("Children")\
.agg(count("Children").alias("Count"), \
     (count("Children")/totalcancelledResort*100).alias("Ratio"))\
.orderBy(col("Ratio").desc())\
     .select("Children","Count",round("Ratio",2).alias("Ratio")).show()

# Improved code
childrenDF.where(col("hotel") == "Resort Hotel")\
.select("Children", "is_canceled")\
.groupby("Children")\
.agg(count(lit(1)).alias("Bookings"),\
    sum("is_canceled").alias("Cancellations"))\
.withColumn("Cancellation Ratio", round(col("Cancellations")/col("Bookings")*100,2))\
.orderBy(col("Cancellation Ratio").desc())\
     .select("Children","Bookings","Cancellations", "Cancellation Ratio").show()

Number of cancellations based on how much in advance the booking was made:

+----------------+-------------+-----+
|    booking_time|Cancellations|Ratio|
+----------------+-------------+-----+
| Within 3 Months|         4244|38.16|
| Within 6 Months|         3075|27.65|
|   Within 1 Year|         3013|27.09|
|   Within 1 Week|          583| 5.24|
|More than 1 Year|          207| 1.86|
+----------------+-------------+-----+

+----------------+--------+-------------+------------------+
|    booking_time|Bookings|Cancellations|Cancellation Ratio|
+----------------+--------+-------------+------------------+
|More than 1 Year|     444|          207|             46.62|
|   Within 1 Year|    7300|         3013|             41.27|
| Within 6 Months|    8216|         3075|             37.43|
| Within 3 Months|   15162|         4244|             27.99|
|   Within 1 Week|    8938|          583|              6.52|
+----------------+--------+-------------+------------------+



Number of cancellations based on deposit type:

+------------+-------------+-----+
|deposit_type|Cancellations|Ratio|
+------------+-------------+-----+
|  No Deposit|         9450|84.97|
|  Non Refund|         1650|14.84|
|  Refundable|           22|  0.2|
+------------+-------------+-----+

+------------+--------+-------------+------------------+
|deposit_type|Bookings|Cancellations|Cancellation Ratio|
+------------+--------+-------------+------------------+
|  Non Refund|    1719|         1650|             95.99|
|  No Deposit|   38199|         9450|             24.74|
|  Refundable|     142|           22|             15.49|
+------------+--------+-------------+------------------+



Number of cancellations, new vs. returning customer:

+------------------+-----+-----+
|     Customer Type|Count|Ratio|
+------------------+-----+-----+
|      New Customer|11011| 99.0|
|Returning Customer|  111|  1.0|
+------------------+-----+-----+

+------------------+--------+-------------+------------------+
|     Customer Type|Bookings|Cancellations|Cancellation Ratio|
+------------------+--------+-------------+------------------+
|      New Customer|   38282|        11011|             28.76|
|Returning Customer|    1778|          111|              6.24|
+------------------+--------+-------------+------------------+



Number of cancellations, traveling for leisure vs. work:

+--------------+-----+-----+
|Booking Reason|Count|Ratio|
+--------------+-----+-----+
|      Personal|10715|96.34|
|          Work|  407| 3.66|
+--------------+-----+-----+

+--------------+--------+-------------+------------------+
|Booking Reason|Bookings|Cancellations|Cancellation Ratio|
+--------------+--------+-------------+------------------+
|      Personal|   36952|        10715|              29.0|
|          Work|    3108|          407|              13.1|
+--------------+--------+-------------+------------------+



Number of cancellations, traveling with children vs. not:

+---------+-----+-----+
| Children|Count|Ratio|
+---------+-----+-----+
|  No Kids| 9725|87.44|
|With Kids| 1397|12.56|
+---------+-----+-----+

+---------+--------+-------------+------------------+
| Children|Bookings|Cancellations|Cancellation Ratio|
+---------+--------+-------------+------------------+
|With Kids|    3929|         1397|             35.56|
|  No Kids|   36131|         9725|             26.92|
+---------+--------+-------------+------------------+

